In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
import numpy as np
import pandas as pd
import os

#Whole Set of Features
data = pd.read_csv("../input/musicfeatures/data.csv")

#MFCCS Set of Features
data_mfccs = pd.read_csv("../input/mfccs-of-1000-songs-across-10-genres/data_mfccs.csv")

#Wave-related Set of Features
data_others = pd.read_csv("../input/tempobeat-analysis-of-1000-songs-10-genres/data_others.csv")

data = data.values
data_mfccs = data_mfccs.values
data_others = data_others.values

temp = data[:,1:]
all_features = temp[:,:28]
mfccs_features = data_mfccs[:,:20]

z = []

for x in range(100):
    z.append(0)
for x in range(100):
    z.append(1)
for x in range(100):
    z.append(2)
for x in range(100):
    z.append(3)
for x in range(100):
    z.append(4)
for x in range(100):
    z.append(5)
for x in range(100):
    z.append(6)
for x in range(100):
    z.append(7)
for x in range(100):
    z.append(8)
for x in range(100):
    z.append(9)


random_guess = np.random.randint(10, size=1000)

z = np.array(z)
print('\tAccuracy of Random Operation:', accuracy_score(z, random_guess, normalize=True)) 
scaler = StandardScaler(with_mean=True, with_std=True)
scaled_data = scaler.fit_transform(all_features)

In [ ]:
genre_labels = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
             
cm = confusion_matrix(z, random_guess)

genres = sn.heatmap(cm,linewidths=.5, annot=True, fmt="d", xticklabels=genre_labels, yticklabels=genre_labels, cmap="BuPu")# font size
genres = genres.set_title('Confusion Matrix - np.randint 0-9')
print(genres)

In [ ]:
from sklearn.linear_model import LogisticRegression
train, test, train_labels, test_labels = train_test_split(scaled_data, z, test_size=0.33, random_state=42)

LG = LogisticRegression(max_iter=10000,solver='newton-cg', multi_class='multinomial').fit(train, train_labels)
pred_train = LG.predict(train)
pred_test = LG.predict(test)
pred_all = LG.predict(scaled_data)

print('\tAccuracy of LogisticRegression-Train:', accuracy_score(train_labels, pred_train, normalize=True))
print('\tAccuracy of LogisticRegression-Test:', accuracy_score(test_labels, pred_test, normalize=True))
print('\tAccuracy of LogisticRegression-All:', accuracy_score(z, pred_all, normalize=True))

In [ ]:
import matplotlib.pyplot as plt

cm = confusion_matrix(pred_test, test_labels)

genres = sn.heatmap(cm,linewidths=.5, annot=True, fmt="d", xticklabels=genre_labels, yticklabels=genre_labels, cmap="BuPu")# font size
genres = genres.set_title('Logistic Regression w/ Softmax - All Features')
print(genres)

In [ ]:
train, test, train_labels, test_labels = train_test_split(scaled_data,z,test_size=0.33,random_state=0)

model = OneVsRestClassifier(LinearSVC(multi_class='ovr', max_iter=10000))
model.fit(train, train_labels)

pred_train = model.predict(train)
pred_test = model.predict(test)
pred_all = model.predict(scaled_data)

print('\tAccuracy of OneVsRest-Train:', accuracy_score(train_labels, pred_train, normalize=True))
print('\tAccuracy of OneVsRest-Test:', accuracy_score(test_labels, pred_test, normalize=True))
print('\tAccuracy of OneVsRest-All:', accuracy_score(z, pred_all, normalize=True))

In [ ]:
cm = confusion_matrix(pred_test, test_labels)
genres = sn.heatmap(cm,linewidths=.5, annot=True, fmt="d", xticklabels=genre_labels, yticklabels=genre_labels, cmap="BuPu")# font size
genres.set_title('OneVsRest Classifier - All Features')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train, test, train_labels, test_labels = train_test_split(scaled_data,z,test_size=0.33,random_state=42)

model = RandomForestClassifier(n_estimators = 1000)
model.fit(train, train_labels)

pred_train = model.predict(train)
pred_test = model.predict(test)
pred_all = model.predict(scaled_data)

print('\tAccuracy of Random Forest-train:', accuracy_score(train_labels, pred_train, normalize=True))
print('\tAccuracy of Random Forest-Test:', accuracy_score(test_labels, pred_test, normalize=True))
print('\tAccuracy of Random Forest-All:', accuracy_score(z, pred_all, normalize=True))

In [ ]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],axis=0)
indices = np.argsort(importances)[::-1]

plt.figure()
plt.title("Feature importances - MFCC Features")
plt.bar(range(scaled_data.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(scaled_data.shape[1]), indices)
plt.xlim([-1, scaled_data.shape[1]])
plt.show()

In [ ]:
cm = confusion_matrix(test_labels, pred_test)
genres = sn.heatmap(cm,linewidths=.5, annot=True, fmt="d", xticklabels=genre_labels, yticklabels=genre_labels, cmap="BuPu")# font size
genres.set_title('Random Forest - All Features')